In [12]:
import torch
from PIL import Image
import numpy as np
import torchvision.transforms.functional as TF
import cv2
from transformers import GLPNFeatureExtractor, GLPNForDepthEstimation

In [3]:
model_name = "vinvino02/glpn-nyu"
feature_extractor = GLPNFeatureExtractor.from_pretrained(model_name)
model = GLPNForDepthEstimation.from_pretrained(model_name)

In [15]:
image_path = "/home/alper/Downloads/room-4779953_960_720.jpg"
image = Image.open(image_path)

In [16]:
inputs = feature_extractor(images=image, return_tensors="pt")

In [17]:
with torch.no_grad():
    outputs = model(**inputs)

In [18]:
predicted_depth = outputs["predicted_depth"].squeeze()
resized_depth = TF.resize(predicted_depth.unsqueeze(0), image.size[::-1], interpolation=Image.BICUBIC).squeeze()
output = resized_depth.cpu().numpy()
formatted = (output * 255 / np.max(output)).astype('uint8')
depth = Image.fromarray(formatted)
depth.show()

/usr/local/lib/python3.11/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


In [19]:
colored = cv2.applyColorMap(formatted, cv2.COLORMAP_RAINBOW)
colored_depth = Image.fromarray(colored)
colored_depth.show()